In [1]:
%reload_ext autoreload
%autoreload 2
import os,sys
sys.path.insert(1, os.path.join(sys.path[0],
                                '..', 'module'))
import wiki
import pickle
import numpy as np
import gensim.models as gm
import gensim.utils as gu
import gensim.corpora as gc
import gensim.matutils as gmat
import matplotlib.pyplot as plt
import sklearn.metrics.pairwise as smp

In [2]:
path_base = '/Users/harangju/Developer/data/wiki/dumps/'
name_xml = 'enwiki-20190801-pages-articles-multistream.xml.bz2'
name_index = 'enwiki-20190801-pages-articles-multistream-index.txt.bz2'
path_xml = path_base + name_xml
path_index = path_base + name_index
dump = wiki.Dump(path_xml, path_index)

### Test big model
See `preparation/build-tfidf-model.ipynb`.

In [3]:
path_save = '/Users/harangju/Developer/data/wiki/models/'
tfidf = gu.SaveLoad.load(path_save + 'tfidf.model')
dct = pickle.load(open(path_save + 'dict.model','rb'))

In [4]:
dump.load_page('Physics')
words = gu.simple_preprocess(dump.page.strip_code())

Dump: Loading index...
Dump: Loaded.


In [5]:
tfidf[dct.doc2bow(words)][:5]

[(4, 0.0030784267402274444),
 (5, 0.2103100457446028),
 (12, 0.0333281420135526),
 (13, 0.2007292418185778),
 (17, 0.04902932080705284)]

### Weight graph

In [39]:
topic = 'earth science'
network = wiki.Net()
network.load_graph('/Users/harangju/Developer/data/wiki/graphs/dated/'+topic+'.gexf')
network.graph.nodes

NodeView(('Planetary science', 'Earth science', 'Lithosphere', 'Hydrosphere', 'Biosphere', 'Holism', 'Geology', 'Geography', 'Chronology', 'Physics', 'Chemistry', 'Biology', 'Mathematics', 'Science', 'Planetary geology', 'Geochemistry', 'Geophysics', 'Oceanography', 'Hydrology', 'Glaciology', 'Life', 'Human geography', 'Physical geography', 'Geosphere', 'Atmospheric sciences', 'Geochronology', 'Plate tectonics', 'Paleoclimatology', 'Petrology', 'Environmental geology', "Earth's magnetic field", 'Geodynamics', 'Magma', 'Magnetosphere', 'Geodesy', 'Crust (geology)', 'Antarctic convergence', 'Atmospheric chemistry', 'Meteorology', 'Volcanology', 'Climatology', 'Atmospheric physics', 'Biogeography', 'Cartography', "Earth's spheres", 'Chemical oceanography', 'Earth (planet)', 'Erosion', 'Cryosphere', 'Mineralogy', 'Dynamo theory', 'Economic geology', 'Structural geology', 'Soil science', 'Edaphology', 'Engineering geology', 'Hydrogeology', 'Geomorphology', 'Environmental science', 'Limnolog

In [40]:
network.graph['Biosphere']

AtlasView({'Earth science': {'id': '26', 'weight': 1.0}, 'Geography': {'id': '27', 'weight': 1.0}, 'Environmental geology': {'id': '28', 'weight': 1.0}, 'Gaia hypothesis': {'id': '29', 'weight': 1.0}, 'Pedosphere': {'id': '30', 'weight': 1.0}, 'Physical geography': {'id': '31', 'weight': 1.0}})

In [41]:
n1 = 'Biosphere'
n2 = 'Earth science'
p1 = gu.simple_preprocess(dump.load_page(n1).strip_code())
p2 = gu.simple_preprocess(dump.load_page(n2).strip_code())
p1[:5], p2[:5]

(['thumb', 'px', 'false', 'color', 'composite'],
 ['earth', 'science', 'or', 'geoscience', 'includes'])

In [42]:
b1 = tfidf[dct.doc2bow(p1)]
b2 = tfidf[dct.doc2bow(p2)]
b1[:3], b2[:3]

([(5, 0.13155321660272415),
  (9, 0.005814052937824806),
  (12, 0.021480370659743037)],
 [(4, 0.018341276283594058),
  (5, 0.12114910825706296),
  (9, 0.005201260490809376)])

In [43]:
v = gmat.corpus2csc([b1, b2])
v[:,0], v[:,1]

(<2050373x1 sparse matrix of type '<class 'numpy.float64'>'
 	with 660 stored elements in Compressed Sparse Column format>,
 <2050373x1 sparse matrix of type '<class 'numpy.float64'>'
 	with 605 stored elements in Compressed Sparse Column format>)

In [44]:
smp.cosine_similarity(X=v[:,0].transpose(), Y=v[:,1].transpose())

array([[0.41726377]])

#### Function

In [48]:
def set_weights(graph, dump, model, dct):
    nodes = list(graph.nodes)
    bows = [model[dct.doc2bow(gu.simple_preprocess(dump.load_page(page).strip_code()))]
            for page in nodes]
    vecs = gmat.corpus2csc(bows)
    for n1, n2 in graph.edges:
        v1 = vecs[:,nodes.index(n1)].transpose()
        v2 = vecs[:,nodes.index(n2)].transpose()
        graph[n1][n2]['weight'] = smp.cosine_similarity(X=v1, Y=v2)

In [49]:
network.graph['Biosphere']

AtlasView({'Earth science': {'id': '26', 'weight': 1.0}, 'Geography': {'id': '27', 'weight': 1.0}, 'Environmental geology': {'id': '28', 'weight': 1.0}, 'Gaia hypothesis': {'id': '29', 'weight': 1.0}, 'Pedosphere': {'id': '30', 'weight': 1.0}, 'Physical geography': {'id': '31', 'weight': 1.0}})

In [50]:
set_weights(network.graph, dump, tfidf, dct)

In [51]:
network.graph['Biosphere']

AtlasView({'Earth science': {'id': '26', 'weight': array([[0.41726377]])}, 'Geography': {'id': '27', 'weight': array([[0.29487946]])}, 'Environmental geology': {'id': '28', 'weight': array([[0.19913704]])}, 'Gaia hypothesis': {'id': '29', 'weight': array([[0.28702049]])}, 'Pedosphere': {'id': '30', 'weight': array([[0.28738003]])}, 'Physical geography': {'id': '31', 'weight': array([[0.2802678]])}})

In [55]:
network.graph['Chemistry']

AtlasView({'Earth science': {'id': '72', 'weight': array([[0.25026822]])}, 'Science': {'id': '73', 'weight': array([[0.44314548]])}, 'Physics': {'id': '74', 'weight': array([[0.3909709]])}, 'Atmospheric chemistry': {'id': '75', 'weight': array([[0.48846208]])}, 'Atmospheric physics': {'id': '76', 'weight': array([[0.24918699]])}, 'Mineralogy': {'id': '77', 'weight': array([[0.36392381]])}, 'Soil science': {'id': '78', 'weight': array([[0.14720482]])}, 'Environmental science': {'id': '79', 'weight': array([[0.27778063]])}, 'Geochemistry': {'id': '80', 'weight': array([[0.45309575]])}, 'Limnology': {'id': '81', 'weight': array([[0.23635818]])}, 'Oceanography': {'id': '82', 'weight': array([[0.29344839]])}, 'Paleoceanography': {'id': '83', 'weight': array([[0.23278909]])}, 'Pedosphere': {'id': '84', 'weight': array([[0.36510943]])}, 'Petrology': {'id': '85', 'weight': array([[0.17507917]])}})